In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
#read the related data 
df= pd.read_csv("xxxxx.csv",parse_dates = ["date"])
df.head()

In [ ]:
#Fill NA values with 0 
df = df.fillna(0)

In [ ]:
#set date column as datetime
df["date"] = pd.to_datetime(df["date"])
df.info()

In [ ]:
#convert all possible dtypes into float or integer if any other exists
df["xxxxx"] = df["xxxxx"].astype(int)
df.info()

In [ ]:
#drop unrelated columns or columns include dtype object
df.drop(columns=["xxxxx","xxxxx","xxxxx"], inplace = True, axis=1)

In [ ]:
#define one product df if there exists more than one product
product = 'productname'
df_prod = df[df['product_name'] == product].sort_values('date')

#make date the index
df_prod = df_prod.set_index('date')  # make date the index

In [ ]:
#define target and exogenous factors that affect the Y variable
#use this if your df consists of only one product
target = df["target"]
exog_features = ["xxxx", "xxxx", "xxxx"]
exog = df[exog_features]

#use this if you convert your df into one product df with code before
endog = df_prod['target']
exog_cols = ["xxxx", "xxxx", "xxxx"]
exog = df_prod[exog_cols]

In [ ]:
# Split train/test (last 30 days as test)
#Use only related code according to your df

#for one product df 
train_target = target[:-30]
test_target = target[-30:]
train_exog = exog[:-30]
test_exog = exog[-30:]

#for converted one product df
endog = df_prod['target']
exog = df_prod[exog_cols]

train_endog, test_endog = endog[:-30], endog[-30:]
train_exog, test_exog = exog[:-30], exog[-30:]

In [ ]:
#Construct SARIMAX model 
#Use only related code according to your df

#for one product df
model_sarimax = SARIMAX(
    train_target,
    exog=train_exog,
    order=(1,1,1),
    seasonal_order=(1,1,1,7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_fit = model_sarimax.fit(disp=False)
print(model_fit.summary())

#for converted one product df
model_sarimax = SARIMAX(train_endog,
                exog=train_exog,
                order=(1,1,1),
                seasonal_order=(1,1,1,7),
                enforce_stationarity=False,
                enforce_invertibility=False)

sarimax_res = model_sarimax.fit(disp=False)

In [ ]:
#Forecast
#Use only related code according to your df

#for one product df
forecast = model_fit.get_forecast(steps=30, exog=test_exog)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

#for converted one product df
# Forecast 7 days recursively
horizon = 7
preds = []
history_endog = train_endog.copy()
history_exog = train_exog.copy()

preds = sarimax_res.get_forecast(steps=len(test_endog), exog=test_exog)
y_pred = preds.predicted_mean  # now y_pred.shape == test_endog.shape

In [ ]:
#Calculate RMSE value
#Use only related code according to your df

#for one product df
for target in targets:
    rmse = np.sqrt(mean_squared_error(test_target, forecast_mean))
    print(f"Test RMSE: {rmse:.2f}")

#for converted one product df
rmse = np.sqrt(mean_squared_error(test_endog, y_pred))
print(f"SARIMAX RMSE: {rmse:.2f}")

In [ ]:
#compare actual and predicted values

#for one product df
results = pd.DataFrame({
    "actual": test_target,
    "forecast": forecast_mean
})
print(results.head(15))

#for converted one product df
for i in range(horizon):
    rmse = np.sqrt(mean_squared_error(y_test_actual[:, i], y_pred[:, i]))
    print(f"RMSE t+{i+1}: {rmse:.2f}")

In [ ]:
#Visualize the final result

#for one product df
plt.figure(figsize=(12,5))
plt.plot(test_target.index, test_target, label="Actual")
plt.plot(test_target.index, forecast_mean, label="Forecast")
plt.fill_between(test_target.index, forecast_ci.iloc[:,0], forecast_ci.iloc[:,1], color='pink', alpha=0.3)
plt.legend()
plt.show()

#for converted one product df
# Plot
plt.figure(figsize=(12,5))
plt.plot(test_endog.index, test_endog.values, linestyle='--', label='Actual')
plt.plot(test_endog.index, y_pred.values, label='SARIMAX Predicted')
plt.legend()
plt.show()